In [11]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to load keypoints and corresponding labels
def load_data(dataset_dir, max_sequence_length=30):
    keypoints = []
    labels = []
    label_map = {}
    label_counter = 0
    
    # Iterate through all the categories (Seasons, Summer, Winter, etc.)
    for category in os.listdir(dataset_dir):
        category_path = os.path.join(dataset_dir, category)
        if os.path.isdir(category_path):
            if category not in label_map:
                label_map[category] = label_counter
                label_counter += 1
            label = label_map[category]

            # Iterate through all videos in each category
            for video in os.listdir(category_path):
                video_path = os.path.join(category_path, video)
                if video.endswith('.npy'):
                    keypoints_data = np.load(video_path)
                    if keypoints_data.size > 0:
                        keypoints.append(keypoints_data)
                        labels.append(label)

    # Pad sequences to ensure uniform length
    keypoints = pad_sequences(keypoints, maxlen=max_sequence_length, padding='post', truncating='post')

    # Flatten the keypoints: [num_frames, num_keypoints * 4]
    # Assuming there are 33 keypoints with 4 values (x, y, z, visibility)
    keypoints = keypoints.reshape(keypoints.shape[0], keypoints.shape[1], 33 * 4)

    return np.array(keypoints), np.array(labels), label_map

# Function to preprocess the data
def preprocess_data(keypoints, labels):
    # Normalize keypoints (optional)
    keypoints = keypoints / np.max(keypoints)  # Normalize keypoints to [0, 1]

    # One-hot encode the labels
    labels = to_categorical(labels)

    return keypoints, labels

# Build the LSTM model
def build_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=input_shape))  # LSTM layer
    model.add(LSTM(64))  # Another LSTM layer
    model.add(Dense(num_classes, activation='softmax'))  # Output layer (softmax for multi-class classification)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Main function to load data, train, and save the model
def train_lstm_model(dataset_dir, model_save_path='sign_language_model.h5'):
    # Load the data
    keypoints, labels, label_map = load_data(dataset_dir)

    # Preprocess the data
    keypoints, labels = preprocess_data(keypoints, labels)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(keypoints, labels, test_size=0.2, random_state=42)

    # Build the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])  # (sequence_length, num_keypoints * 4)
    num_classes = len(label_map)
    model = build_lstm_model(input_shape, num_classes)

    # Train the model
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

    # Save the trained model
    model.save(model_save_path)
    print(f"Model saved at {model_save_path}")

# Run the training process
dataset_dir = 'Keypoints/Seasons'  # Path to the directory containing the keypoints
train_lstm_model(dataset_dir)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - accuracy: 0.1404 - loss: 1.8360 - val_accuracy: 0.1176 - val_loss: 1.7849
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1864 - loss: 1.7890 - val_accuracy: 0.1176 - val_loss: 1.8033
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2309 - loss: 1.7890 - val_accuracy: 0.1176 - val_loss: 1.8274
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1664 - loss: 1.8045 - val_accuracy: 0.1176 - val_loss: 1.8299
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1898 - loss: 1.7878 - val_accuracy: 0.1176 - val_loss: 1.8340
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.2240 - loss: 1.7897 - val_accuracy: 0.1176 - val_loss: 1.8302
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2006 - loss: 1.7902 - val_accuracy: 0.1765 - val_loss: 1.8211
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2080 - loss: 1.7862 - val_accuracy: 0.2353 - val_loss

Model saved at sign_language_model.h5
